# Snippet Creator and Hacking Space 

Hacking space. 

Use snippets.py (previously dm.py) or whatever for the code... 

## Provenance

* dm.ipynb renamed snippet_runner.ipynb to coordinate with snippets.py
* Moved into agg project, untracked June 28, 2022
* Created new as Learning_SLY June 23, 2022

In [ ]:
# see the parser
%run aggregate/parser.py

In [ ]:
agg.parser.UnderwritingParser.enhance_debugfile()

In [ ]:
logger_level(30)
%run -i snippets.py

In [ ]:
# save grammar 
agg.grammar()

In [ ]:
test = build('agg TEST dfreq [1 2 3 4] dsev [1 2 5 10]')
test

In [ ]:
test.report_df

# NewCo Re Capital Model

## Effect on the tail of adding cat...really depends on the relative vol of non-cat.

In [ ]:
logger_level(30)
%run -i snippets.py 
# %run -i snippets.py parse_tests

In [ ]:
porta = build('''
port newco
    agg NonCatAndReserves 1 claim sev gamma 27000 cv 0.05 fixed
    agg Cat               1 claim sev lognorm  500 cv 0.500 fixed
''', log2=17, bs=1, padding=1)

portb = build('''
port newco
    agg NonCatAndReserves 1 claim sev gamma  27000 cv .01 fixed
    agg Cat               1 claim sev lognorm  500 cv 0.500 fixed
''', log2=17, bs=1, padding=2)

In [ ]:
porta.plot()

In [ ]:
portb.plot()

In [ ]:
%%sf 1 2 8 8 
porta.biv_contour_plot(f, ax0, 0, 40e3, 1e3, cmap='viridis')
portb.biv_contour_plot(f, ax1, 0, 40e3, 1e3, cmap='viridis')

## Cat / Non-Cat Capital Model Case: New Syntax

In [ ]:
logger_level(30)
%run -i snippets.py

In [ ]:
prog_noncat = 'agg NC 1 claim sev gamma 6840 cv 0.037 # 19500 fixed'
noncat = build(prog_noncat)
noncat.report_df

In [ ]:
prog_cat_gross = 'agg CAT [26.144 2] claims sev lognorm [40 100] cv [1.7 2.5] mixed gamma 0.4'
cat = build(prog_cat_gross, log2=18, bs=1)
cat.report_df

In [ ]:
prog_cat_net = 'agg CAT [26.144 2] claims sev lognorm [40 100] cv [1.7 2.5] occurrence net of 1000 x 400 mixed gamma 0.4 aggregate net of 1325 xs 2000'
net = build(prog_cat_net, log2=18, bs=1)
net.report_df

In [ ]:
cat.q(0.9),cat.q(0.99), cat.q(0.996), cat.q(0.999), net.q(0.9), net.q(0.99), net.q(0.996), net.q(0.999)

In [ ]:
# blend calibration
# A- bond, default 5 bps, spread 195 bps over treasuriesw
# most recent cat bond EL 0.009, spread 0.053
# cat bonds average el 4.2% at 8.5%
# s_values = [0.0005, 0.009, 0.042]
# y = np.array([0.0195, 0.042, 0.085])
s_values = [0.0005, 0.009]
y = np.array([0.0195, 0.042])
gs_values = y / (1+y)

In [ ]:
a, prem = catco.pricing_summary.loc[['a', 'P'], 'gross']
a, prem

In [ ]:
ans

In [ ]:
ans = catco.gross.calibrate_blends(a, prem, s_values, gs_values, debug=True)

In [ ]:
catco.dist_dict[list( catco.dist_dict.keys() )[-1]].plot(both=False)

In [ ]:
%%sf 1 4 4 4 
for k, v in catco.blend_distortions.items():
    ax = next(axi)
    v.plot(ax=ax, both=False, lw=1)
    ax.set(title=k)
ax = next(axi)
catco.blend_d.plot(ax=ax, both=False, lw=1)
ax.set(title=
      'Blend-d')

In [ ]:
%%sf 1 4 4 4 
g = agg.Distortion('ph', 0.5)
for k, v in ans[3].items():
    ax = next(axi)
    g.g = v
    g.plot(ax=ax, both=False, lw=1)
    ax.set(title=k)

In [ ]:
catco = cs.CaseStudy()
catco.factory(case_id='CatCo',
                   case_name='CatCo Capital Model',
                   case_description='CatCo capital model calibrated to S&P',
                   a_distribution       = prog_noncat,
                   b_distribution_gross = prog_cat_gross,
                   b_distribution_net   = prog_cat_net,
                   reg_p=0.99995,      # backed into number so the net capital is correct
                   roe=0.085,
                   d2tc=0.281118428,    # to get about the right amount of debt
                   f_discrete=False, 
                   s_values=s_values,
                   gs_values=gs_values,
                   bs=1,
                   log2=18,
                   padding=1)
catco.to_json()

In [ ]:
catco.full_monty()

In [ ]:
catco.show_extended_graphs()

In [ ]:
mrr = cs.ManualRenderResults(catco)

In [ ]:
# custom for this analysis
mrr.render_custom('F', 'A', 'N', 'M', 'O', 'V', 'E', 'Z*', 'B')

In [ ]:
catco.show_extended_graphs()

# Define severity as a mixture?

In [ ]:
out = build('sev Atest [10 30 100 300 1000] @ expon 1 wts [.4 .2 .2 .1 .1]')

In [ ]:
out

In [ ]:
a = build('agg Amixed 10 claim sev.Atest poisson', log2=16, bs=.5)
a

In [ ]:
a.report_df

In [ ]:
a.plot()